In [1]:
from mhr.utils.utils import *
from tqdm import tqdm

from transformers import  HfArgumentParser
from tqdm import tqdm
from dataclasses import dataclass, field


origin_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/sharegpt4v_train/ft/sharegpt4v_mix665k_cap23k_coco-ap9k_lcs3k_sam9k_div2k.json"
token_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/sharegpt4v_train/ft/sharegpt4v_ft_tokens.jsonl"
object_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/sharegpt4v_train/ft/sharegpt4v_ft_dino_objects.jsonl"
object_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/sharegpt4v_train/ft/sharegpt4v_ft_dino_objects.jsonl"
what_word_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/sharegpt4v_train/ft/sharegpt4v_ft_llama_what_words.jsonl"


output_path = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/sharegpt4v_train/ft/origin_data_full_info/share_ft_dinoobj.json"

origin_data = load_json_file(origin_file)
token_data = process_jsonl(token_file)
object_data = process_jsonl(object_file)
what_data = process_jsonl(what_word_file)

id_key = "new_idx"

object_to_item_dict = {str(item[id_key]):str2list(item.get("objects",item.get("outputs",""))) for item in object_data}
token_to_item_dict = {str(item[id_key]):str2list(item["tokens"]) for item in token_data}
what_word_to_item_dict = {str(item[id_key]):str2list(item.get("objects",item.get("outputs",""))) for item in what_data}

def build_full_info_file():
    output_data=[]
    for idx,item in enumerate(tqdm(origin_data)):
        new_idx = str(idx)
        # id = str(item["id"])
        id = new_idx
        tokens = token_to_item_dict.get(str(id),[])
        objects = object_to_item_dict.get(str(id),[])
        co_occurrence = objects_to_co_occurrence(objects)
        what_words = what_word_to_item_dict.get(str(id),[])
        statistics = dict(token=tokens,object=objects,co_occurrence=co_occurrence,what_word=what_words)
        item["statistics"]=statistics
        item["new_idx"] = new_idx
        output_data.append(item)
    return output_data

def objects_to_co_occurrence(objects):
    co_occurrence = []
    objects = list(set(objects))
    if len(objects) < 2:
        return []
    for i in range(len(objects)):
        for j in range(i+1,len(objects)):
            co_occurrence.append(get_two_words(objects[i],objects[j]))
    return co_occurrence


/mnt/petrelfs/songmingyang/anaconda3/envs/lt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
id_key = "new_idx"
def check_new_index_mapping():
    object_to_item_dict_c = {str(item[id_key]):item for item in object_data}
    token_to_item_dict_c = {str(item[id_key]):item for item in token_data}
    what_word_to_item_dict_c = {str(item[id_key]):item for item in what_data}
    object_err_cnt,token_err_cnt,what_word_err_cnt = 0,0,0
    for idx,item in enumerate(tqdm(origin_data)):
        id = str(idx)
        ori_id = str(item["id"])
        ori_image = item.get("image",None)
        object_item = object_to_item_dict_c.get(id)
        if object_item and  ori_id != object_item["id"]:
            object_err_cnt+=1
        token_item = token_to_item_dict_c.get(id)
        if token_item and ( ori_id != str(token_item["id"]) or (ori_image and ori_image != token_item["image"] )):
            print(f"ori_id:{ori_id},token_item:{token_item}")
            print(f"item:{item}")
            token_err_cnt+=1
            # break
        what_word_item = what_word_to_item_dict_c.get(id)
        if what_word_item and (ori_id != what_word_item["id"] or (ori_image and ori_image != what_word_item["image"])):
            what_word_err_cnt+=1
    print(f"object_err_cnt:{object_err_cnt},token_err_cnt:{token_err_cnt},what_word_err_cnt:{what_word_err_cnt}")

In [23]:
check_new_index_mapping()

100%|██████████| 665058/665058 [00:01<00:00, 414301.61it/s]

object_err_cnt:0,token_err_cnt:0,what_word_err_cnt:0


In [2]:
output_data = build_full_info_file()


  0%|          | 0/665058 [00:00<?, ?it/s]

100%|██████████| 665058/665058 [00:07<00:00, 83785.51it/s] 


In [3]:
write_json_file(output_data,output_path)